In [ ]:
import requests, time
import pandas as pd

In [ ]:
query = "perovskite solar halide passivation"
base_url = f"https://api.crossref.org/works"

rows_per_request = 1000
all_dois = []

most_recent_year = 2024
for i in range(most_recent_year, most_recent_year - 5, -1):
    offset = 0
    while offset < 10000:
        url = f"{base_url}?query={query}&rows={rows_per_request}&offset={offset}&filter=from-pub-date:{i},until-pub-date:{i}"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed request at offset {offset}: {response.status_code}")
            break

        data = response.json()
        items = data['message']['items']

        if not items:  # Stop when there are no more results
            break

        # Extract DOIs
        for item in items:
            if 'DOI' in item:
                all_dois.append(item)

        print(f"Fetched {len(items)} records (Offset: {offset})")
        offset += rows_per_request  # Move to the next batch
        time.sleep(1)  # Be polite and avoid rate limits

In [ ]:
df = pd.DataFrame(all_dois)
df["year"] = df["created"].apply(lambda x: x["date-parts"][0][0])
df = df[['DOI', 'URL', 'year', 'title', 'publisher']]
df.head()

In [ ]:
df.to_csv('/data/crossref_data.csv', index=False)